# M2177.003100 Deep Learning <br> Final Proejct: Text to Image Synthesis (Tensorflow)

**In order to unify a scoring tool (fairness), we decided to use one evaluation code. The evaluation code for pytorch and tensorflow users is the same. **

### Submitting your work:
<font color=red>**DO NOT clear the R-precision score **</font> so that TAs can grade the result.

In [2]:
import tensorflow as tf
import numpy as np

import sys, os
sys.path.append('..')

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

from miscc.config import cfg, cfg_from_file

cfg_from_file('../cfg/eval_birds.yml')

In [3]:
def cosine_similarity(v1, v2):
    """
    Returns cosine similarity between v1 and v2
    """
    cost = tf.reduce_sum(tf.multiply(v1, v2), 1) / (tf.sqrt(tf.reduce_sum(tf.multiply(v1, v1), 1)) * tf.sqrt(tf.reduce_sum(tf.multiply(v2, v2), 1)))
    return cost

In [4]:
path = cfg.R_PRECISION_FILE
print(path)

'''
Size of input features
true_cnn_features = [cfg.NUM_BATCH_FOR_TEST, cfg.TRAIN.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM]
true_rnn_features = [cfg.NUM_BATCH_FOR_TEST, cfg.TRAIN.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM]
wrong_rnn_features = [cfg.NUM_BATCH_FOR_TEST, cfg.WRONG_CAPTION, cfg.TRAIN.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM]
'''

if path.endswith('.npz'):
    f = np.load(path)
    true_cnn_features, true_rnn_features, wrong_rnn_features = f['true_cnn'], f['true_rnn'], f['wrong_rnn']
    f.close()

print(true_cnn_features.shape)
print(true_rnn_features.shape)
print(wrong_rnn_features.shape)

cfg.NUM_BATCH_FOR_TEST = true_cnn_features.shape[0]
cfg.BATCH_SIZE = true_cnn_features.shape[1]
cfg.TEXT.EMBEDDING_DIM = true_cnn_features.shape[2]

r_precision.npz
(916, 32, 128)
(916, 32, 128)
(916, 9, 32, 128)


In [5]:
t_real_image = tf.placeholder('float32', [cfg.NUM_BATCH_FOR_TEST * cfg.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM], name='real_image')
t_real_cap = tf.placeholder('float32', [cfg.NUM_BATCH_FOR_TEST * cfg.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM], name='real_caption')
t_wrong_cap = tf.placeholder('float32', [cfg.NUM_BATCH_FOR_TEST * cfg.BATCH_SIZE, cfg.TEXT.EMBEDDING_DIM], name='wrong_caption')

In [6]:
true_cnn_features = np.reshape(true_cnn_features, (-1, cfg.TEXT.EMBEDDING_DIM))
true_rnn_features = np.reshape(true_rnn_features, (-1, cfg.TEXT.EMBEDDING_DIM))

true_cnn_true_rnn = cosine_similarity(t_real_image, t_real_cap)
true_cnn_wrong_rnn = cosine_similarity(t_real_image, t_wrong_cap)


with tf.Session() as sess:
    cs_b_real_real = sess.run(true_cnn_true_rnn, feed_dict={t_real_image:true_cnn_features, t_real_cap:true_rnn_features})
    cs_b_real_wrong = []
    for i in range(cfg.WRONG_CAPTION):
        cs_b_real_wrong.append(sess.run(true_cnn_wrong_rnn, feed_dict={t_real_image: true_cnn_features,
                                                                  t_wrong_cap: np.reshape(wrong_rnn_features[:, i, :, :], (-1, cfg.TEXT.EMBEDDING_DIM))}))

    cs_b_real_wrong = np.asarray(cs_b_real_wrong)

    total_cs = tf.concat((np.expand_dims(cs_b_real_real, 0), cs_b_real_wrong), axis=0)

    ranked = tf.argmax(total_cs, 0)
    correct_ranking = tf.equal(ranked, 0)

    r_precision = tf.reduce_mean(tf.cast(correct_ranking, tf.float32))

    print('R-precision: ' + str(sess.run(r_precision)))
    # R-precision: 0.43538597 - Batchsize = 16, 50 epoch
    # R-precision: 0.47864357 - 190 epoch, 32 batchsize
    # R-precision: 0.41112855 - 300 epoch => checked
    # R-precision: 0.33859852 - 350 epoch => checked
    # R-precision: 0.32672626 - 360 epoch
    # R-precision: 0.48556906 - 370 epoch
    # R-precision: 0.42218205 - 380 epoch

R-precision: 0.42218205
